<a href="https://colab.research.google.com/github/Shubham-Sahay/BQML_Hackathon/blob/main/BQ_Hackathon_Aggregated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# Author : Lavi Nigam, ML Engineering @ Google 
# Linkedin: https://www.linkedin.com/in/lavinigam/ 

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/lavinigam-gcp/BQML_Hackathon/blob/main/LTV_Prediction/BQ_Hackathon_StarterNotebook.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/lavinigam-gcp/BQML_Hackathon/blob/main/LTV_Prediction/BQ_Hackathon_StarterNotebook.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
</table>

Before starting the notebook, make sure you have two things: 

1) Acknowledging that you finished the pre-requisite [here](https://machinehack.com/hackathons/google_cloud_bigquery_ltv_prediction_challenge/overview?prerequisites=true)

2) Project ID from Google Cloud. 

Do not proceed further wihout these steps. 

Set your project ID here.

In [2]:
PROJECT_ID = ""
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "mh-bq-hackathon-375011"  # @param {type:"string"}

PROJECT_ID

'mh-bq-hackathon-375011'

### Authenticate your Google Cloud account

***Ensure you are logged out of all google (Gmail) accounts except the one you created for this Hackathon. We recommend using a different browser or making sure you have opened Colab using Hackathon's newly created Gmail account..***

In [3]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

In [4]:
# Importing some important libraries that will be used during the notebook
import pandas as pd
import plotly.express as px
from google.cloud import bigquery

In [5]:
#Client manages connections to the BigQuery API and helps
#bundle configuration (project, credentials) needed for API requests.
client = bigquery.Client(PROJECT_ID)

# to make sure all columns are displayed while working with dataframe
pd.set_option('display.max_columns', None)

## Assumptions

## Exploratory Data Analysis (EDA)

You can start by defining some essential variables that can change according to your data. It is always better to consider the most recent records from your data as features. For this purpose, you can set the START_DATE and END_DATE based on your data recency.

In this case, the date range is set for 3 months.

In [6]:
MAIN_PROJECT_ID_DATA = "mh-hackathon"
MAIN_DATASET_ID_DATA = "ga4_data" 
MAIN_TABLE_ID_TRAIN  = "ga4_train"
MAIN_TABLE_ID_TEST  = "ga4_test"
START_DATE = "20201101"
END_DATE = "20210131"

You can start the data exploration by returning the first five rows of data.
The data has multiple event tables for each day. So, all the tables (events) could be queried by using events* as the wildcard.

[GA4 Data Export Schema](https://support.google.com/analytics/answer/7029846#zippy=)

Note: BigQuery export, by default, are [date sharded tables](https://cloud.google.com/bigquery/docs/partitioned-tables#dt_partition_shard)

In [ ]:
query = f"""
SELECT
  *
FROM
  `{MAIN_PROJECT_ID_DATA}.{MAIN_DATASET_ID_DATA}.{MAIN_TABLE_ID_TRAIN}`
LIMIT
  5
"""
query_job = client.query(query)
top5_data = query_job.to_dataframe()
top5_data.head()

,user_pseudo_id,event_date,event_timestamp,event_name,event_params,event_previous_timestamp,event_value_in_usd,event_bundle_sequence_id,event_server_timestamp_offset,user_id,privacy_info,user_properties,user_first_touch_timestamp,device,geo,app_info,traffic_source,stream_id,platform,event_dimensions,ecommerce,items,ltv
0,30900021.3233786964,20201111,1605075729011915,user_engagement,"[{'key': 'campaign', 'value': {'string_value':...",<NA>,NaN,7918536415,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1602033447181464,"{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Asia', 'sub_continent': 'Wester...",None,"{'medium': '(none)', 'name': '(direct)', 'sour...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[],8.012165
1,61031472.8710389493,20210117,1610869460342446,scroll,"[{'key': 'session_engaged', 'value': {'string_...",<NA>,NaN,2342622982,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1610867956673433,"{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Asia', 'sub_continent': 'Wester...",None,"{'medium': '(data deleted)', 'name': '(data de...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[],41.952304
2,5790952.3877084543,20201208,1607399935112245,user_engagement,"[{'key': 'campaign', 'value': {'string_value':...",<NA>,NaN,313973033,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1607399913193448,"{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Asia', 'sub_continent': 'Wester...",None,"{'medium': 'organic', 'name': '(organic)', 'so...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[],90.731843
3,84123100.3290360033,20201108,1604864379849130,user_engagement,"[{'key': 'clean_event', 'value': {'string_valu...",<NA>,NaN,156071925,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1604864373827450,"{'category': 'mobile', 'mobile_brand_name': 'A...","{'continent': 'Asia', 'sub_continent': 'Wester...",None,"{'medium': 'referral', 'name': '(referral)', '...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[],8.388783
4,67488528.1173442205,20201223,1608690304951540,session_start,"[{'key': 'ga_session_number', 'value': {'strin...",<NA>,NaN,585568866,<NA>,None,"{'analytics_storage': None, 'ads_storage': Non...",[],1608690304951540,"{'category': 'mobile', 'mobile_brand_name': 'A...","{'continent': 'Asia', 'sub_continent': 'Wester...",None,"{'medium': 'organic', 'name': '(organic)', 'so...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[],48.579845


## Data Preparation

In [7]:
# You can create the dataset through code.
DATASET_NAME = "shubham_dataset"

try:
  dataset = client.create_dataset(DATASET_NAME, timeout=30)  # Make an API request.
  print("Created dataset {}.{}".format(client.project, dataset.dataset_id))
except Exception as e:
  print(e)

409 POST https://bigquery.googleapis.com/bigquery/v2/projects/mh-bq-hackathon-375011/datasets?prettyPrint=false: Already Exists: Dataset mh-bq-hackathon-375011:shubham_dataset


## Event Info Extraction

In [ ]:
query = f"""
CREATE OR REPLACE VIEW
  {DATASET_NAME}.EventView_train AS
SELECT
  user_pseudo_id,
  event_name,
  count(event_timestamp) as eventCount
FROM
  `{MAIN_PROJECT_ID_DATA}.{MAIN_DATASET_ID_DATA}.{MAIN_TABLE_ID_TRAIN}`
GROUP BY
  user_pseudo_id,
  event_name
  """
query_job = client.query(query)

In [ ]:
query = f"""
CREATE OR REPLACE VIEW
  {DATASET_NAME}.EventView_test AS
SELECT
  user_pseudo_id,
  event_name,
  count(event_timestamp) as eventCount
FROM
  `{MAIN_PROJECT_ID_DATA}.{MAIN_DATASET_ID_DATA}.{MAIN_TABLE_ID_TEST}`
GROUP BY
  user_pseudo_id,
  event_name
  """
query_job = client.query(query)

In [ ]:
query = f"""
SELECT
  *
FROM
  `{DATASET_NAME}.EventView_train`
"""
query_job = client.query(query)
eventView_train = query_job.to_dataframe()
eventView_train.head(5)

,user_pseudo_id,event_name,eventCount
0,4511811.3990598489,add_payment_info,14
1,9222870.9095775010,add_payment_info,5
2,41493433.3540028865,add_payment_info,1
3,2964823.8798022419,add_payment_info,15
4,3046093.2167709173,add_payment_info,7


In [ ]:
query = f"""
SELECT
  *
FROM
  `{DATASET_NAME}.EventView_test`
"""
query_job = client.query(query)
eventView_test = query_job.to_dataframe()
eventView_test.head(5)

,user_pseudo_id,event_name,eventCount
0,1055969.5872512303,add_payment_info,2
1,1063363.8922941272,add_payment_info,3
2,1079913.1905515044,add_payment_info,1
3,1219636.7573278529,add_payment_info,5
4,1421856.0079266060,add_payment_info,6


In [ ]:
# Did not created pivot over big query because of reaching query execution limit. Error is mentioned below
# Script expression exceeded evaluation limit of 1048576 bytes at [2:14]
eventViewPivot_train = eventView_train.pivot(index='user_pseudo_id', columns='event_name', values='eventCount').fillna(0).reset_index()
eventViewPivot_train.head()

event_name,user_pseudo_id,add_payment_info,add_shipping_info,add_to_cart,begin_checkout,click,first_visit,page_view,purchase,scroll,select_item,select_promotion,session_start,user_engagement,view_item,view_item_list,view_promotion,view_search_results
0,10001363.4360935308,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0
1,1000223163.8035209215,0,0,0,0,0,1,2,0,1,0,0,1,1,0,0,0,0
2,1000299.7413851356,0,0,0,0,0,1,2,0,0,0,0,1,1,0,0,0,0
3,1000300.3223254235,0,0,0,0,0,1,1,0,0,0,0,2,1,0,0,1,0
4,10004358.0897722689,0,0,0,0,0,1,2,0,1,0,0,1,0,0,0,0,0


In [ ]:
# Did not created pivot over big query because of reaching query execution limit. Error is mentioned below
# Script expression exceeded evaluation limit of 1048576 bytes at [2:14]
eventViewPivot_test = eventView_test.pivot(index='user_pseudo_id', columns='event_name', values='eventCount').fillna(0).reset_index()
eventViewPivot_test.head()

event_name,user_pseudo_id,add_payment_info,add_shipping_info,add_to_cart,begin_checkout,click,first_visit,page_view,purchase,scroll,select_item,select_promotion,session_start,user_engagement,view_item,view_item_list,view_promotion,view_search_results
0,10003031.4607645453,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0
1,10006188.0272495437,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0
2,10007546.3519345336,0,0,0,0,0,1,5,0,4,0,0,1,4,5,0,0,0
3,1000823.8498711409,0,0,0,0,0,1,3,0,1,0,0,2,2,0,0,1,0
4,1000985.4712566084,0,0,0,0,0,1,3,0,0,0,0,1,2,0,0,0,1


In [ ]:
eventViewPivot_train.to_gbq(f'{DATASET_NAME}.EventView_Pivot_train', project_id=PROJECT_ID, if_exists='replace', progress_bar=True)

100%|██████████| 1/1 [00:00<00:00, 1315.65it/s]


In [ ]:
eventViewPivot_test.to_gbq(f'{DATASET_NAME}.EventView_Pivot_test', project_id=PROJECT_ID, if_exists='replace', progress_bar=True)

100%|██████████| 1/1 [00:00<00:00, 1327.73it/s]


## Item Category View

In [11]:
query = f"""
  CREATE OR REPLACE VIEW `{DATASET_NAME}.ItemCategoryView` AS
  WITH itemCat AS(
    SELECT
    DISTINCT(items[SAFE_OFFSET(0)].item_category),
    user_pseudo_id
    FROM `{MAIN_PROJECT_ID_DATA}.{MAIN_DATASET_ID_DATA}.{MAIN_TABLE_ID_TRAIN}`
  )
  SELECT user_pseudo_id,
    CASE
      WHEN item_category LIKE '%Lifestyle%' THEN 'Lifestyle'
      WHEN item_category LIKE '%Men%' THEN 'Men'
      WHEN item_category LIKE '%Kid%' THEN 'Kid'
      WHEN item_category LIKE '%Women%' THEN 'Women'
      WHEN item_category LIKE '%Office%' THEN 'Office'
      WHEN item_category LIKE '%Water Bottles%' THEN 'Water Bottles'
      WHEN item_category LIKE '%Writing%' THEN 'Writing' 
      WHEN item_category LIKE '%Notebooks%' THEN 'Notebooks'
      WHEN item_category LIKE '%New%' THEN 'New'
      ELSE itemCat.item_category
      END AS item_category
  FROM itemCat
  ORDER BY itemCat.item_category DESC
  """
query_job = client.query(query)

In [12]:
# create feature table for training
query = f"""
  CREATE OR REPLACE TABLE `{DATASET_NAME}.ItemCategoryTable` AS
  (
      WITH 
      cte1 AS (
        SELECT 
          user_pseudo_id, 
          ANY_VALUE(item_category) as item_category
        FROM `{DATASET_NAME}.ItemCategoryView`
        GROUP BY user_pseudo_id
      ),
      cte2 AS (
        SELECT user_pseudo_id,
        geo.country as country,
        event_name
        FROM `{MAIN_PROJECT_ID_DATA}.{MAIN_DATASET_ID_DATA}.{MAIN_TABLE_ID_TRAIN}`
        GROUP BY user_pseudo_id
      )
        
    SELECT 
      cte2.user_pseudo_id,
      item_category,
      country,
      event_name
    FROM cte1 
    RIGHT JOIN cte2
    ON cte1.user_pseudo_id = cte2.user_pseudo_id
  )
  """
query_job = client.query(query)
result_df = query_job.to_dataframe()
result_df.head()

""


## Data Preparation

In [ ]:
feature_table="trainData" #table name

query = f"""
CREATE OR REPLACE TABLE
  {DATASET_NAME}.{feature_table} AS
SELECT * except (user_id) FROM
(SELECT
  user_pseudo_id as user_id,
  MAX(CAST(format_date('%m',parse_date("%Y%m%d",event_date)) as INT64)) as month_of_the_year,
  MAX(CAST(format_date('%U',parse_date("%Y%m%d",event_date)) as INT64)) as week_of_the_year,
  MAX(CAST(format_date('%d',parse_date("%Y%m%d",event_date)) as INT64)) as day_of_the_month,
  MAX(CAST(format_date('%w',parse_date("%Y%m%d",event_date)) as INT64)) as day_of_week,
  COUNT(event_name) as event_count,
  SUM(IFNULL(event_value_in_usd, 0)) as total_order_value,
  AVG(ltv) as ltv
FROM
  `{MAIN_PROJECT_ID_DATA}.{MAIN_DATASET_ID_DATA}.{MAIN_TABLE_ID_TRAIN}`
Group by 
user_pseudo_id) as main
INNER JOIN
`{DATASET_NAME}.EventView_Pivot_train` as ev
ON main.user_id=ev.user_pseudo_id
  """
query_job = client.query(query)

In [ ]:
feature_table="testData" #table name

query = f"""
CREATE OR REPLACE TABLE
  {DATASET_NAME}.{feature_table} AS
SELECT * except (user_id) FROM
(SELECT
  user_pseudo_id as user_id,
  MAX(CAST(format_date('%m',parse_date("%Y%m%d",event_date)) as INT64)) as month_of_the_year,
  MAX(CAST(format_date('%U',parse_date("%Y%m%d",event_date)) as INT64)) as week_of_the_year,
  MAX(CAST(format_date('%d',parse_date("%Y%m%d",event_date)) as INT64)) as day_of_the_month,
  MAX(CAST(format_date('%w',parse_date("%Y%m%d",event_date)) as INT64)) as day_of_week,
  COUNT(event_name) as event_count,
  SUM(IFNULL(event_value_in_usd, 0)) as total_order_value
FROM
  `{MAIN_PROJECT_ID_DATA}.{MAIN_DATASET_ID_DATA}.{MAIN_TABLE_ID_TEST}`
Group by 
user_pseudo_id) as main
LEFT JOIN
`{DATASET_NAME}.EventView_Pivot_test` as ev
ON main.user_id=ev.user_pseudo_id
  """
query_job = client.query(query)

## Performing PCA

In [ ]:
query = f"""
CREATE MODEL
  `{DATASET_NAME}.PCA_model`
OPTIONS
  ( MODEL_TYPE='PCA',
    PCA_EXPLAINED_VARIANCE_RATIO=0.99 ) AS
SELECT
  * except (user_pseudo_id, ltv)
FROM
  `{PROJECT_ID}.{DATASET_NAME}.trainData`
"""
print(query)


CREATE MODEL
  `shubham_dataset.PCA_model`
OPTIONS
  ( MODEL_TYPE='PCA',
    PCA_EXPLAINED_VARIANCE_RATIO=0.99 ) AS
SELECT
  * except (user_pseudo_id, ltv)
FROM
  `mh-bq-hackathon-375011.shubham_dataset.trainData`



## Model Building

In [ ]:
# model_name = "dl_model"
# feature_table="trainData"

# dl_query = f"""
# CREATE OR REPLACE MODEL
#   `{DATASET_NAME}.{model_name}` 
#   OPTIONS (
#       MODEL_TYPE = 'DNN_REGRESSOR',
#       DROPOUT = 0.2,
#       EARLY_STOP = TRUE,
#     EARLY_STOP = FALSE,
#     MAX_ITERATIONS = 20,
#     DATA_SPLIT_METHOD = 'AUTO_SPLIT',
#     ENABLE_GLOBAL_EXPLAIN = TRUE,
#     INPUT_LABEL_COLS = ['ltv']
#    ) 
# AS
# SELECT
#   * except (user_pseudo_id)
# FROM
#   `{PROJECT_ID}.{DATASET_NAME}.{feature_table}`
# WHERE
#   ltv IS NOT NULL
# """
# print(dl_query)
# query_job = client.query(dl_query)
# #RUN THIS ON BQ CONSOLE 

In [ ]:
model_name = "customer_ltv_model_LR"


query = f"""
CREATE OR REPLACE MODEL
  `{DATASET_NAME}.{model_name}` 
  OPTIONS (
      MODEL_TYPE = 'LINEAR_REG',
      OPTIMIZE_STRATEGY = 'AUTO_STRATEGY',
    L1_REG = 0.1,
    L2_REG = 0.1,
    EARLY_STOP = TRUE,
    LEARN_RATE_STRATEGY = 'LINE_SEARCH',
    MIN_REL_PROGRESS = 0.01,
    DATA_SPLIT_METHOD = 'AUTO_SPLIT',
    CATEGORY_ENCODING_METHOD = 'DUMMY_ENCODING',
    ENABLE_GLOBAL_EXPLAIN = FALSE,
    INPUT_LABEL_COLS = ['ltv']
   ) 
AS
SELECT
  *
FROM
  ML.PREDICT(
    MODEL {DATASET_NAME}.PCA_model,
    (
      SELECT
        * except (user_pseudo_id)
      FROM
      `{PROJECT_ID}.{DATASET_NAME}.trainData` 
    )
  )
"""
print(query)
# query_job = client.query(query)
#RUN THIS ON BQ CONSOLE 


CREATE OR REPLACE MODEL
  `shubham_dataset.customer_ltv_model_LR` 
  OPTIONS (
      MODEL_TYPE = 'LINEAR_REG',
      OPTIMIZE_STRATEGY = 'AUTO_STRATEGY',
    L1_REG = 0.1,
    L2_REG = 0.1,
    EARLY_STOP = TRUE,
    LEARN_RATE_STRATEGY = 'LINE_SEARCH',
    MIN_REL_PROGRESS = 0.01,
    DATA_SPLIT_METHOD = 'AUTO_SPLIT',
    CATEGORY_ENCODING_METHOD = 'DUMMY_ENCODING',
    ENABLE_GLOBAL_EXPLAIN = FALSE,
    INPUT_LABEL_COLS = ['ltv']
   ) 
AS
SELECT
  *
FROM
  ML.PREDICT(
    MODEL shubham_dataset.PCA_model,
    (
      SELECT
        * except (user_pseudo_id)
      FROM
      `mh-bq-hackathon-375011.shubham_dataset.trainData` 
    )
  )



In [ ]:
model_name = "customer_ltv_model_RF"


query = f"""
CREATE OR REPLACE MODEL
  `{DATASET_NAME}.{model_name}` 
  OPTIONS (
      MODEL_TYPE = 'RANDOM_FOREST_REGRESSOR',
    NUM_PARALLEL_TREE = 100,
    TREE_METHOD = 'AUTO',
    COLSAMPLE_BYTREE = 0.7,
    COLSAMPLE_BYLEVEL = 0.8,
    MIN_TREE_CHILD_WEIGHT = 5,
    MAX_TREE_DEPTH = 10,
    EARLY_STOP = TRUE,
    MIN_REL_PROGRESS = 0.01,
    DATA_SPLIT_METHOD = 'AUTO_SPLIT',
    ENABLE_GLOBAL_EXPLAIN = FALSE,
    INPUT_LABEL_COLS = ['ltv']
   ) 
AS
SELECT
  *
FROM
  ML.PREDICT(
    MODEL {DATASET_NAME}.PCA_model,
    (
      SELECT
        * except (user_pseudo_id)
      FROM
      `{PROJECT_ID}.{DATASET_NAME}.trainData` 
    )
  )
"""
print(query)
# query_job = client.query(query)
#RUN THIS ON BQ CONSOLE 


CREATE OR REPLACE MODEL
  `shubham_dataset.customer_ltv_model_RF` 
  OPTIONS (
      MODEL_TYPE = 'RANDOM_FOREST_REGRESSOR',
    NUM_PARALLEL_TREE = 100,
    TREE_METHOD = 'AUTO',
    COLSAMPLE_BYTREE = 0.7,
    COLSAMPLE_BYLEVEL = 0.8,
    MIN_TREE_CHILD_WEIGHT = 5,
    MAX_TREE_DEPTH = 10,
    EARLY_STOP = TRUE,
    MIN_REL_PROGRESS = 0.01,
    DATA_SPLIT_METHOD = 'AUTO_SPLIT',
    ENABLE_GLOBAL_EXPLAIN = FALSE,
    INPUT_LABEL_COLS = ['ltv']
   ) 
AS
SELECT
  *
FROM
  ML.PREDICT(
    MODEL shubham_dataset.PCA_model,
    (
      SELECT
        * except (user_pseudo_id)
      FROM
      `mh-bq-hackathon-375011.shubham_dataset.trainData` 
    )
  )



In [ ]:
model_name = "customer_ltv_model_AutoML"


query = f"""
CREATE OR REPLACE MODEL `{DATASET_NAME}.{model_name}`
       OPTIONS(model_type='AUTOML_REGRESSOR',
               input_label_cols=['ltv'],
               budget_hours=3)
AS
SELECT
  *
FROM
  ML.PREDICT(
    MODEL {DATASET_NAME}.PCA_model,
    (
      SELECT
        * except (user_pseudo_id)
      FROM
      `{PROJECT_ID}.{DATASET_NAME}.trainData` 
    )
  )
"""
print(query)
# query_job = client.query(query)
#RUN THIS ON BQ CONSOLE 


CREATE OR REPLACE MODEL `shubham_dataset.customer_ltv_model_AutoML`
       OPTIONS(model_type='AUTOML_REGRESSOR',
               input_label_cols=['ltv'],
               budget_hours=3)
AS
SELECT
  *
FROM
  ML.PREDICT(
    MODEL shubham_dataset.PCA_model,
    (
      SELECT
        * except (user_pseudo_id)
      FROM
      `mh-bq-hackathon-375011.shubham_dataset.trainData` 
    )
  )



In [ ]:
ml_evaluate_query = f"""
SELECT
  *
FROM
  ML.EVALUATE(MODEL `{DATASET_NAME}.{model_name}`,
    (
    SELECT
      *
    FROM
    ML.PREDICT(
      MODEL {DATASET_NAME}.PCA_model,
      (
        SELECT
          *
        FROM
        `{PROJECT_ID}.{DATASET_NAME}.trainData` 
      )
  )))
"""
query_job = client.query(ml_evaluate_query)
ml_info_df = query_job.to_dataframe()
ml_info_df

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,65.732897,10801.809906,2.545865,48.456258,0.001389,0.001531


## Prediction on Test & Submission

In [ ]:
model_name

'customer_ltv_model_LR'

In [ ]:
prediction_data_table_name = "model_prediction_ltv_test"
query = f"""
CREATE OR REPLACE TABLE
  {DATASET_NAME}.{prediction_data_table_name} AS
SELECT
  *
FROM
  ML.PREDICT(MODEL `{DATASET_NAME}.{model_name}`,
    (
    SELECT
      *
    FROM
      ML.PREDICT(
        MODEL {DATASET_NAME}.PCA_model,
        (
          SELECT
            *
          FROM
          `{DATASET_NAME}.testData` 
    )
  )
    ))
"""
print(query)
query_job = client.query(query)


CREATE OR REPLACE TABLE
  shubham_dataset.model_prediction_ltv_test AS
SELECT
  *
FROM
  ML.PREDICT(MODEL `shubham_dataset.customer_ltv_model_LR`,
    (
    SELECT
      *
    FROM
      ML.PREDICT(
        MODEL shubham_dataset.PCA_model,
        (
          SELECT
            *
          FROM
          `shubham_dataset.testData` 
    )
  )
    ))



In [ ]:
query = f"""
SELECT
  *
FROM
  {PROJECT_ID}.{DATASET_NAME}.{prediction_data_table_name}
LIMIT 5
"""
# print(query)
query_job = client.query(query)
predict_data = query_job.to_dataframe()
predict_data.head()

,predicted_ltv,principal_component_1,principal_component_2,principal_component_3,principal_component_4,principal_component_5,principal_component_6,principal_component_7,principal_component_8,principal_component_9,principal_component_10,principal_component_11,principal_component_12,principal_component_13,principal_component_14,principal_component_15,principal_component_16,principal_component_17,principal_component_18,user_pseudo_id
0,64.520155,0.226721,1.508955,-1.713554,0.646030,1.476732,-0.728493,0.416446,0.135765,-0.044129,-3.028377,0.140180,0.167257,-0.378288,-0.409191,0.357710,-0.122761,0.049702,0.042009,64417055.8162935475
1,70.471852,5.245266,-2.830880,-2.334250,6.500799,1.998744,5.537649,-4.602381,3.646465,1.205656,0.313332,-1.046022,-0.882550,-0.841401,-0.037517,-2.006132,0.019992,-0.787019,-0.474218,33249122.0720596771
2,65.151896,-0.748163,1.872846,-0.640674,0.792330,1.690410,-0.640271,0.403053,-0.065114,-0.200555,-0.428507,0.182346,-0.524961,0.555255,0.298487,0.031048,-0.032251,0.076395,0.071731,20351734.5335122794
3,65.151896,-0.748163,1.872846,-0.640674,0.792330,1.690410,-0.640271,0.403053,-0.065114,-0.200555,-0.428507,0.182346,-0.524961,0.555255,0.298487,0.031048,-0.032251,0.076395,0.071731,80131197.6959290120
4,65.151896,-0.748163,1.872846,-0.640674,0.792330,1.690410,-0.640271,0.403053,-0.065114,-0.200555,-0.428507,0.182346,-0.524961,0.555255,0.298487,0.031048,-0.032251,0.076395,0.071731,5489574.7785488344


In [ ]:
query = f"""
SELECT
  user_pseudo_id
FROM
  {PROJECT_ID}.{DATASET_NAME}.testData
"""
# print(query)
query_job = client.query(query)
idData = query_job.to_dataframe()
idData.head()

,user_pseudo_id
0,60383283.3591233159
1,2492500.1425673190
2,26265970.9544747517
3,1255322.2973246181
4,4406329.3988718578


In [ ]:
query = f"""
SELECT
  predicted_ltv
FROM
  {PROJECT_ID}.{DATASET_NAME}.{prediction_data_table_name}
"""
# print(query)
query_job = client.query(query)
submission_data = query_job.to_dataframe()
submission_data.head()

,predicted_ltv
0,64.520155
1,70.471852
2,65.151896
3,65.151896
4,65.151896


In [ ]:
## Create Submission 
submission = pd.concat([idData, submission_data], axis=1)
submission.columns = ['user_pseudo_id',"predicted_avg_ltv"]
submission.to_csv("submission.csv",index=False)
submission.head()

,user_pseudo_id,predicted_avg_ltv
0,60383283.3591233159,64.520155
1,2492500.1425673190,70.471852
2,26265970.9544747517,65.151896
3,1255322.2973246181,65.151896
4,4406329.3988718578,65.151896


In [ ]:
#make sure before submission that your shape is (26760, 2). Do not submit unless its of same shape. 
submission.shape

(26760, 2)